In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import zipfile
import scipy.stats as st
import math
from datetime import datetime

# suppress futurewarnings from statistical packages
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.ticker as mticker
from tabulate import tabulate
import plotly
import plotly.express as px

Unzipping csv from kaggel

In [63]:
#read data

zip_file = "../Dataset/AB_US_2020.zip"
try:
    with zipfile.ZipFile(zip_file, 'r') as z:
        z.extractall("../Dataset/unZipped/")
        print("Extracted all")
except:
    print("Invalid file")
    
airbnbDS = pd.read_csv("../Dataset/unZipped/AB_US_2020.csv")
airbnbDS

Extracted all


C:\Users\clineman\Anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning:

Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.



,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne,NaN,28804,35.651460,-82.627920,Private room,60,1,138,16/02/20,1.14,1,0,Asheville
1,80905,French Chic Loft,427027,Celeste,NaN,28801,35.597790,-82.555400,Entire home/apt,470,1,114,07/09/20,1.03,11,288,Asheville
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,Lisa,NaN,28801,35.606700,-82.555630,Entire home/apt,75,30,89,30/11/19,0.81,2,298,Asheville
3,155305,Cottage! BonPaul + Sharky's Hostel,746673,BonPaul,NaN,28806,35.578640,-82.595780,Entire home/apt,90,1,267,22/09/20,2.39,5,0,Asheville
4,160594,Historic Grove Park,769252,Elizabeth,NaN,28801,35.614420,-82.541270,Private room,125,30,58,19/10/15,0.52,1,0,Asheville
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226025,45506143,"DC Hidden In Plain ""Site""",25973146,Marci,NaN,"Downtown, Chinatown, Penn Quarters, Mount Vern...",38.903880,-77.029730,Entire home/apt,104,1,0,NaN,NaN,2,99,Washington D.C.
226026,45511428,DC 3 BR w/ screen porch 3 blck to metro w/ par...,231133074,Thomas,NaN,"Brookland, Brentwood, Langdon",38.920820,-76.990980,Entire home/apt,151,2,0,NaN,NaN,1,300,Washington D.C.
226027,45514685,Charming Penthouse Apt w/ Rooftop Terrace in L...,33758935,Bassem,NaN,"Shaw, Logan Circle",38.911170,-77.033540,Entire home/apt,240,2,0,NaN,NaN,1,173,Washington D.C.
226028,45516412,Adams Morgan/Nat'l Zoo 1 BR Apt #32,23193071,Michael,NaN,"Kalorama Heights, Adams Morgan, Lanier Heights",38.926630,-77.044360,Entire home/apt,60,21,0,NaN,NaN,5,362,Washington D.C.


In [64]:
airbnbDS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226030 entries, 0 to 226029
Data columns (total 17 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              226030 non-null  int64  
 1   name                            226002 non-null  object 
 2   host_id                         226030 non-null  int64  
 3   host_name                       225997 non-null  object 
 4   neighbourhood_group             110185 non-null  object 
 5   neighbourhood                   226030 non-null  object 
 6   latitude                        226030 non-null  float64
 7   longitude                       226030 non-null  float64
 8   room_type                       226030 non-null  object 
 9   price                           226030 non-null  int64  
 10  minimum_nights                  226030 non-null  int64  
 11  number_of_reviews               226030 non-null  int64  
 12  last_review     

In [65]:
airbnbDS.isnull ().sum()

id                                     0
name                                  28
host_id                                0
host_name                             33
neighbourhood_group               115845
neighbourhood                          0
latitude                               0
longitude                              0
room_type                              0
price                                  0
minimum_nights                         0
number_of_reviews                      0
last_review                        48602
reviews_per_month                  48602
calculated_host_listings_count         0
availability_365                       0
city                                   0
dtype: int64

In [66]:
airbnbDS.shape

(226030, 17)

Going to drop neighbourhood_group because of half of the values are null. For both name and host name I will fill the empty slots with "N/A", there is very few missing and I do not think name will be very important to regesion on price. For all null values in reviews_per_month will be replaced with 0. 

In [67]:
airbnbDS = airbnbDS.drop("neighbourhood_group", axis = 1)


In [68]:
airbnbDS = airbnbDS.fillna(value= {'name': 'N/A', 'host_name':'N/A', 'reviews_per_month': 0})

In [69]:
airbnbDS.isnull ().sum()

id                                    0
name                                  0
host_id                               0
host_name                             0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       48602
reviews_per_month                     0
calculated_host_listings_count        0
availability_365                      0
city                                  0
dtype: int64

Dropping all null last review rows because they there is not enough null rows to effect the data set and there is no justifiable number to fill the column with. 

In [70]:
airbnbDS = airbnbDS.dropna()

In [71]:
airbnbDS.isnull ().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
city                              0
dtype: int64

Now that all of the columns are Null free we will begin encoding the data.

#### Encoding

In [72]:
airbnbDS.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne,28804,35.65146,-82.62792,Private room,60,1,138,16/02/20,1.14,1,0,Asheville
1,80905,French Chic Loft,427027,Celeste,28801,35.59779,-82.55540,Entire home/apt,470,1,114,07/09/20,1.03,11,288,Asheville
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,Lisa,28801,35.60670,-82.55563,Entire home/apt,75,30,89,30/11/19,0.81,2,298,Asheville
3,155305,Cottage! BonPaul + Sharky's Hostel,746673,BonPaul,28806,35.57864,-82.59578,Entire home/apt,90,1,267,22/09/20,2.39,5,0,Asheville
4,160594,Historic Grove Park,769252,Elizabeth,28801,35.61442,-82.54127,Private room,125,30,58,19/10/15,0.52,1,0,Asheville


In [73]:
airbnbDS.room_type.value_counts()

Entire home/apt    122196
Private room        51028
Shared room          2777
Hotel room           1427
Name: room_type, dtype: int64

Encoding room types so it can be used in the final regession.

In [74]:
room_ID_dic = {'Entire home/apt': 1, 'Private room': 2,'Shared room': 3, 'Hotel room': 4}

airbnbDS['room_type_ID'] = airbnbDS['room_type'].apply(lambda x: room_ID_dic[x])

In [75]:
airbnbDS

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city,room_type_ID
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne,28804,35.65146,-82.62792,Private room,60,1,138,16/02/20,1.14,1,0,Asheville,2
1,80905,French Chic Loft,427027,Celeste,28801,35.59779,-82.55540,Entire home/apt,470,1,114,07/09/20,1.03,11,288,Asheville,1
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,Lisa,28801,35.60670,-82.55563,Entire home/apt,75,30,89,30/11/19,0.81,2,298,Asheville,1
3,155305,Cottage! BonPaul + Sharky's Hostel,746673,BonPaul,28806,35.57864,-82.59578,Entire home/apt,90,1,267,22/09/20,2.39,5,0,Asheville,1
4,160594,Historic Grove Park,769252,Elizabeth,28801,35.61442,-82.54127,Private room,125,30,58,19/10/15,0.52,1,0,Asheville,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225936,45326130,Modern and cozy home located in Washington DC,55489711,Amir,"Union Station, Stanton Park, Kingman Park",38.90101,-77.00283,Entire home/apt,144,1,1,18/09/20,1.00,1,328,Washington D.C.,1
225950,45349877,Penthouse w/ Patio ‚òÜ Capitol Hill Condo ‚òÜ ...,3850096,Ije,"Capitol Hill, Lincoln Park",38.88703,-77.00586,Entire home/apt,132,2,1,16/09/20,1.00,8,162,Washington D.C.,1
225955,45352724,MODERN ‚òÜ Well-Located Shaw Town Home ‚òÜ 2BR...,3850096,Ije,"Howard University, Le Droit Park, Cardozo/Shaw",38.91626,-77.02074,Entire home/apt,112,2,1,13/09/20,1.00,8,171,Washington D.C.,1
225964,45385834,Brand new modern apartment with private entry,16561471,Victor,"Brightwood Park, Crestwood, Petworth",38.94358,-77.01283,Entire home/apt,78,1,1,19/09/20,1.00,1,75,Washington D.C.,1


In [76]:
airbnbDS.isnull ().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
city                              0
room_type_ID                      0
dtype: int64

encoding city to prevent too many dummy variables and make this column usefull in the final regresion

In [77]:
airbnbDS['city'].unique()

array(['Asheville', 'Austin', 'Boston', 'Broward County', 'Cambridge',
       'Chicago', 'Clark County', 'Columbus', 'Denver', 'Hawaii',
       'Jersey City', 'Los Angeles', 'Nashville', 'New Orleans',
       'New York City', 'Oakland', 'Pacific Grove', 'Portland',
       'Rhode Island', 'Salem', 'San Clara Country', 'San Diego',
       'San Francisco', 'San Mateo County', 'Santa Cruz County',
       'Seattle', 'Twin Cities MSA', 'Washington D.C.'], dtype=object)

In [78]:
city_ID_dic = {'Asheville' : 1, 'Austin' : 2, 'Boston' :3, 'Broward County' :4, 'Cambridge':5,
       'Chicago':6, 'Clark County':7, 'Columbus':8, 'Denver':9, 'Hawaii':10,
       'Jersey City':11, 'Los Angeles':12, 'Nashville':13, 'New Orleans':14,
       'New York City':15, 'Oakland':16, 'Pacific Grove':17, 'Portland':18,
       'Rhode Island':19, 'Salem':20, 'San Clara Country':21, 'San Diego':22,
       'San Francisco':23, 'San Mateo County':24, 'Santa Cruz County':25,
       'Seattle':26, 'Twin Cities MSA':27, 'Washington D.C.':28}


airbnbDS['city_ID'] = airbnbDS['city'].apply(lambda x: city_ID_dic[x])

In [79]:
airbnbDS

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city,room_type_ID,city_ID
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne,28804,35.65146,-82.62792,Private room,60,1,138,16/02/20,1.14,1,0,Asheville,2,1
1,80905,French Chic Loft,427027,Celeste,28801,35.59779,-82.55540,Entire home/apt,470,1,114,07/09/20,1.03,11,288,Asheville,1,1
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,Lisa,28801,35.60670,-82.55563,Entire home/apt,75,30,89,30/11/19,0.81,2,298,Asheville,1,1
3,155305,Cottage! BonPaul + Sharky's Hostel,746673,BonPaul,28806,35.57864,-82.59578,Entire home/apt,90,1,267,22/09/20,2.39,5,0,Asheville,1,1
4,160594,Historic Grove Park,769252,Elizabeth,28801,35.61442,-82.54127,Private room,125,30,58,19/10/15,0.52,1,0,Asheville,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225936,45326130,Modern and cozy home located in Washington DC,55489711,Amir,"Union Station, Stanton Park, Kingman Park",38.90101,-77.00283,Entire home/apt,144,1,1,18/09/20,1.00,1,328,Washington D.C.,1,28
225950,45349877,Penthouse w/ Patio ‚òÜ Capitol Hill Condo ‚òÜ ...,3850096,Ije,"Capitol Hill, Lincoln Park",38.88703,-77.00586,Entire home/apt,132,2,1,16/09/20,1.00,8,162,Washington D.C.,1,28
225955,45352724,MODERN ‚òÜ Well-Located Shaw Town Home ‚òÜ 2BR...,3850096,Ije,"Howard University, Le Droit Park, Cardozo/Shaw",38.91626,-77.02074,Entire home/apt,112,2,1,13/09/20,1.00,8,171,Washington D.C.,1,28
225964,45385834,Brand new modern apartment with private entry,16561471,Victor,"Brightwood Park, Crestwood, Petworth",38.94358,-77.01283,Entire home/apt,78,1,1,19/09/20,1.00,1,75,Washington D.C.,1,28


#### Feature Engeneer

creating a new column of when last date reviewed was to try and help better train the model

In [80]:
airbnbDS['last_review'] = pd.to_datetime(airbnbDS['last_review'])

In [81]:
airbnbDS.info()

airbnbDS['days_since_last_review'] = datetime.now() - airbnbDS['last_review']



airbnbDS

airbnbDS.isnull ().sum()

airbnbDS.info()

airbnbDS['days_since_last_review'] = airbnbDS['days_since_last_review'].dt.days

airbnbDS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177428 entries, 0 to 226002
Data columns (total 18 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   id                              177428 non-null  int64         
 1   name                            177428 non-null  object        
 2   host_id                         177428 non-null  int64         
 3   host_name                       177428 non-null  object        
 4   neighbourhood                   177428 non-null  object        
 5   latitude                        177428 non-null  float64       
 6   longitude                       177428 non-null  float64       
 7   room_type                       177428 non-null  object        
 8   price                           177428 non-null  int64         
 9   minimum_nights                  177428 non-null  int64         
 10  number_of_reviews               177428 non-null  int64  